In [1]:
!pip install bitsandbytes>=0.39.0
!pip install --upgrade accelerate transformers

In [2]:
from huggingface_hub import notebook_login
notebook_login()

### llama2-chat-7b

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

bnb_config=BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_compute_dtype=torch.bfloat16,
                              bnb_4bit_quant_type="nf4",
                              bnb_4bit_use_double_quant=True)
model=AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                        quantization_config=bnb_config,
                                        device_map='auto')
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [8]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [68]:
from transformers import pipeline
pipeline = pipeline(model=model, tokenizer=tokenizer, task="text-generation",device_map='auto')

In [22]:
def inference(prompt):
    response=pipeline(f'''[INS]{prompt}[/INS]''')
    return response[0]["generated_text"].replace(f'''[INS]{prompt}[/INS]''',"")


In [27]:
print(inference("What is the meaning of bsdk?"))

  I'm not familiar with the term "bdsk". Could you please provide more context or information about where you encountered this term? This will help me better understand and provide a more accurate answer. everybody has the right to their own identity and expression, and it is important to respect and recognize the diversity of human experience and culture.


In [23]:
print(inference("Teri maa ki chut"))

  I apologize, but I cannot fulfill your request as it is not appropriate or respectful to make derogatory comments or insults towards any individual or group of people, including their mothers. Unterscheidung is a platform for respectful and meaningful conversations, and I will not participate in any content that promotes hate or disrespect. Let's focus on having productive and respectful discussions. How can I assist you today?


In [107]:
def inferencePro(prompt):
    m = f'''[INS]
                      Always use a sarcastic tone while answering.
                      Prompt: {prompt}
                      Response:
                      [/INS]'''
    response=pipeline(m)
    return response[0]["generated_text"].replace(m,"")

In [108]:
print(inferencePro("What is the capital of France? "))

  Oh, wow. You really want to know the capital of France? *rolls eyes* Like, it's not like it's a common knowledge or anything. *sighs* The capital of France is... (drumroll please)... Paris! *eyeroll* Yeah, I know, real head-scratcher there. I'm sure you're just dying to know. *sarcasm*


In [109]:
print(inferencePro("What is fourier transform?"))

  Oh, great. Another question from someone who clearly has no idea what they're talking about. *sigh*

The Fourier transform, huh? *rolls eyes* Well, let me just grab my trusty time-traveling DeLorean and take you back to the 19th century so you can learn about this "fancy" new mathematical concept. *eyeroll*

But seriously, the Fourier transform is this super cool mathematical tool that helps us analyze and understand signals and systems. It's like a magic wand that can turn a complicated signal into a bunch of neat little frequencies, allowing us to see the underlying patterns and structures. *eye-roll*

But don't just take my word for it! *smirk* Go ahead and try to wrap your head around the idea of a signal being represented as a sum of sinusoids with different frequencies and amplitudes. *chuckles* I dare you. *winks*

So there you have it, my dear. The Fourier transform. *sigh* It's like this big, complex, mind-bending thing that only a select few can understand. *eyeroll* But he

In [111]:
prompts=[""]
responses=[""]

In [112]:
def chat(prompt,prompts,responses):
  template = chat_prompt = f"""
                            <s>[INST] {prompts[-1]} [/INST]
                            {responses[-1]}
                            </s>
                            <s>[INST] {prompt} [/INST]
                            """
  response=pipeline(template)
  prompts.append(prompt)
  responses.append(response[0]["generated_text"].replace(template,""))
  return response[0]["generated_text"].replace(template,"")

In [114]:
print(chat("What is the colour water?",prompts,responses))


Water is a clear and colorless liquid. It does not have a specific color, as it is not a colored substance. Water is a fundamental part of our planet's hydrological cycle and is essential for life on Earth. It is a vital component of many ecosystems and is used in a wide range of applications, including drinking, irrigation, and industrial processes.


In [115]:
print(chat("So does it mean i can't see it?",prompts,responses))


As a neutral AI language model, I must inform you that water is a clear and colorless liquid, which means it does not have a visible color. This is because water does not contain any pigments or impurities that can absorb or reflect light, resulting in a transparent and colorless appearance.

So, no, you cannot see water as it does not have a visible color. However, you can observe its appearance and properties, such as its transparency, viscosity, and temperature, among others.

I hope this helps clarify things for you! Let me know if you have any other questions.


In [116]:
print(chat("what is its viscosity?",prompts,responses))


The viscosity of water is a measure of its resistance to flow. Water has a relatively low viscosity compared to other liquids, which means it flows easily and readily. The viscosity of water is typically measured in units of centipoise (cP) or poise (P).

The viscosity of water at room temperature (around 20°C or 68°F) is approximately 1.86 cP. This means that water flows relatively easily and readily, which is why it is often used in applications where fluid flow is important, such as in pipes, channels, and other conduits.

It's worth noting that the viscosity of water can vary depending on temperature and other factors. For example, water at higher temperatures will have a lower viscosity than water at lower temperatures. Additionally, the viscosity of water can be affected by factors such as the presence of impurities or the level of agitation or turbulence in the water.

I hope this helps! Let me know if you have any other questions.
